## ここはいろいろ実験的なコードを書くところとする。

### 7/20 pandas_profilingの実験

In [2]:
import pandas as pd
import numpy as np
genba = pd.read_csv("data/train_genba.tsv", sep='\t')
goto = pd.read_csv("data/train_goto.tsv",sep='\t')
df = goto.merge(genba, how='left', on='pj_no')
df.to_csv("data/tmp.csv")

In [3]:
import pandas_profiling as pdp

In [4]:
pdp.ProfileReport(df)

Number of variables,158
Number of observations,6461
Total Missing (%),41.4%
Total size in memory,7.8 MiB
Average record size in memory,1.2 KiB
Numeric,47
Categorical,102
Boolean,1
Date,0
Text (Unique),1
Rejected,7


## 7/15追加
- 現場中で最小面積の号棟にフラグを付ける
- 現場中に一つの号棟しかない場合に、その号棟にフラグを付ける

In [5]:
import pandas as pd
import numpy as np
genba = pd.read_csv("data/train_genba.tsv", sep='\t')
goto = pd.read_csv("data/train_goto.tsv",sep='\t')
df = goto.merge(genba, how='left', on='pj_no')

In [14]:
df['genba_su']=df.apply(lambda x: x['tateuri_su']+x['tochiuri_su']+x['joken_su'],axis=1)
df['only_one']=df.apply(lambda x: 1 if x['genba_su'] == 0 else 0 , axis=1)
df['min_tc_mseki_goto'] = df.apply(lambda x: 1 if x['tc_mseki']==x['tc_mseki_min_hb'] and x['only_one']==0 else 0, axis=1)

In [19]:
df[:,df['genba_su']==1]

TypeError: '(slice(None, None, None), 0        True
1        True
2        True
3        True
4       False
5       False
6       False
7        True
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17       True
18       True
19      False
20      False
21      False
22      False
23      False
24       True
25       True
26       True
27      False
28      False
29       True
        ...  
6431    False
6432    False
6433    False
6434    False
6435    False
6436    False
6437    False
6438    False
6439    False
6440    False
6441    False
6442    False
6443     True
6444     True
6445    False
6446    False
6447    False
6448    False
6449    False
6450    False
6451    False
6452    False
6453    False
6454    False
6455    False
6456    False
6457    False
6458    False
6459    False
6460    False
Name: genba_su, Length: 6461, dtype: bool)' is an invalid key

### 7/14 magutchとniwasakiを標準化する

In [16]:
import pandas as pd
import numpy as np
train = pd.read_csv("data/processed_train_goto_x_v8.csv")

In [18]:
from sklearn.preprocessing import StandardScaler
type(train['magutchi'])

pandas.core.series.Series

In [19]:
sc=StandardScaler()

In [20]:
mg = sc.fit_transform(train['magutchi'].values.reshape(-1,1))

In [30]:
train['magutchi'].std()

4.721680065559077

In [31]:
train['magutchi'].mean()

8.92837486457202

In [32]:
std=train['magutchi'].std()
mean=train['magutchi'].mean()
train['std_magutchi']=train['magutchi'].apply(lambda x: (x-mean)/std)

In [34]:
train['std_magutchi'].head()

0    0.205780
1    0.078706
2    0.459926
3    0.332853
4    0.968220
Name: std_magutchi, dtype: float64

In [35]:
train['magutchi'].head()

0     9.9
1     9.3
2    11.1
3    10.5
4    13.5
Name: magutchi, dtype: float64

In [36]:
train['std_magutchi'].mean()

6.0513083134030414e-15

In [37]:
train['std_magutchi'].std()

1.0

### 7/14 住所や路線が精度に影響しているかを確認する

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/train_data_error.csv")

In [2]:
df1 = df[['id','keiyaku_pr','pred_keiyaku_pr','error']]

In [3]:
genba = pd.read_csv("data/train_genba.tsv", sep='\t')
train = pd.read_csv("data/train_goto.tsv",sep='\t')

df2 = df1.merge(train, how='left', on='id')
df3 = df2.merge(genba, how='left', on='pj_no')


In [4]:
df3.to_csv("data/tmp.csv")

### 7/9 levelplan補完コード

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/test_goto.tsv", sep='\t')
df['levelplan'] = df.apply(lambda x : '土地売り' if type(x['levelplan']) == float and x['tt_mseki']==0 else x['levelplan'], axis=1)
df.to_csv("data/tmp.csv")
df['levelplan']=df['levelplan'].fillna('2F/4LDK')
df['levelplan'].isnull().sum()

0

In [22]:
df['levelplan']=df['levelplan'].fillna('2F/4LDK')

In [23]:
df['levelplan'].isnull().sum()

0

## 7/7 実験用コード
setsudo_hi列の改善
'南＋北西'といった複数の組み合わせを、'南'と'北西'の二つに分ける

In [3]:
# データを呼んでカラムを抽出し、横に並んだカラムを縦に並べてファイル出力したい
import pandas as pd
df = pd.read_csv("data/processed_train_goto_x.csv")
cols = pd.DataFrame(df.columns.values)
cols.to_csv('data/columns.csv')

#### setsudo_hiを処理するコード

In [31]:
import pandas as pd
df = pd.read_csv("data/train_goto.tsv", sep='\t')

In [32]:
# '+'と'＋'が混ざっているので統一する
df['setsudo_hi']=df['setsudo_hi'].replace({'北+東':'北＋東', '北+西':'北＋西', '北西+南西':'北西＋南西','南東+南西':'南東＋南西'})
df = pd.merge(df, pd.get_dummies(df[['setsudo_hi']],drop_first = False), left_index = True, right_index = True )

In [33]:
def process_setsudo_hi_2(df,i):
    if df.at[i,'setsudo_hi_北東＋北西']==1:
        df.at[i,'setsudo_hi_北東']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_北東＋南東']==1:
        df.at[i,'setsudo_hi_北東']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_北東＋南西']==1:
        df.at[i,'setsudo_hi_北東']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_北西＋北東']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_北西＋南東']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_北西＋南西']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_北西＋東']==1:
        df.at[i,'setsudo_hi_北西']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_北＋北東']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_北＋北西']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_北＋南']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_南']=1
    elif df.at[i,'setsudo_hi_北＋南東']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_北＋南西']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_北＋東']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_北＋西']==1:
        df.at[i,'setsudo_hi_北']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_南東＋北']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南東＋北東']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_南東＋北西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_南東＋南西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_南東＋北西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_南東＋南西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_南東＋東']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_南東＋西']==1:
        df.at[i,'setsudo_hi_南東']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_南西＋北']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南西＋北']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南西＋北東']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_南西＋北西']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_南西＋南東']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_南西＋北']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南西＋東']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_南西＋西']==1:
        df.at[i,'setsudo_hi_南西']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_南＋北']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_南＋北東']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_南＋北西']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_南＋南東']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_南＋南西']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_南＋東']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_南＋西']==1:
        df.at[i,'setsudo_hi_南']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_東＋北']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_東＋南']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_南']=1
    elif df.at[i,'setsudo_hi_東＋南東']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_東＋南西']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_東＋西']==1:
        df.at[i,'setsudo_hi_東']=1
        df.at[i,'setsudo_hi_西']=1
    elif df.at[i,'setsudo_hi_西＋北']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_北']=1
    elif df.at[i,'setsudo_hi_西＋北東']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_北東']=1
    elif df.at[i,'setsudo_hi_西＋北西']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_北西']=1
    elif df.at[i,'setsudo_hi_西＋東']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_東']=1
    elif df.at[i,'setsudo_hi_西＋南東']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_南東']=1
    elif df.at[i,'setsudo_hi_西＋南西']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_南西']=1
    elif df.at[i,'setsudo_hi_西＋東']==1:
        df.at[i,'setsudo_hi_西']=1
        df.at[i,'setsudo_hi_東']=1
    return


In [37]:
for i in range(len(df)):
    process_setsudo_hi_2(df,i)

In [38]:
df.to_csv('data/tmp.csv')

In [27]:
# 地道に'＋'を分解する
    def process_setsudo_hi(df):
        if df['setsudo_hi_北東＋北西']==1:
            df['setsudo_hi_北東']=1
            df['setsudo_hi_北西']=1
        elif df['setsudo_hi_北東＋南東']==1:
            df['setsudo_hi_北東']=1
            df['setsudo_hi_南東']=1
        elif df['setsudo_hi_北東＋南西']==1:
            df['setsudo_hi_北東']=1
            df['setsudo_hi_南西']=1
        elif df['setsudo_hi_北西＋北東']==1:
            df['setsudo_hi_北西']=1
            df['setsudo_hi_北東']=1
        elif df['setsudo_hi_北西＋南東']==1:
            df['setsudo_hi_北西']=1
            df['setsudo_hi_南東']=1
        elif df['setsudo_hi_北西＋南西']==1:
            df['setsudo_hi_北西']=1
            df['setsudo_hi_南西']=1
        elif df['setsudo_hi_北西＋東']==1:
            df['setsudo_hi_北西']=1
            df['setsudo_hi_東']=1
        elif df['setsudo_hi_北＋北東']==1:
            df['setsudo_hi_北']=1
            df['setsudo_hi_北東']=1
        elif df['setsudo_hi_北＋北西']==1:
            df['setsudo_hi_北']=1
            df['setsudo_hi_北西']=1
        elif df['setsudo_hi_北＋南']==1:
            df['setsudo_hi_北']=1
            df['setsudo_hi_南']=1
        elif df['setsudo_hi_北＋南東']==1:
            df['setsudo_hi_北']=1
            df['setsudo_hi_南東']=1
        elif df['setsudo_hi_北＋南西']==1:
            df['setsudo_hi_北']=1
            df['setsudo_hi_南西']=1
        elif df['setsudo_hi_北＋東']==1:
            df['setsudo_hi_北']=1
            df['setsudo_hi_東']=1
        elif df['setsudo_hi_北＋西']==1:
            df['setsudo_hi_北']=1
            df['setsudo_hi_西']=1
        elif df['setsudo_hi_南東＋北']==1:
            df['setsudo_hi_南東']=1
            df['setsudo_hi_北']=1
        elif df['setsudo_hi_南東＋北東']==1:
            df['setsudo_hi_南東']=1
            df['setsudo_hi_北東']=1
        elif df['setsudo_hi_南東＋北西']==1:
            df['setsudo_hi_南東']=1
            df['setsudo_hi_北西']=1
        elif df['setsudo_hi_南東＋南西']==1:
            df['setsudo_hi_南東']=1
            df['setsudo_hi_南西']=1
        elif df['setsudo_hi_南東＋北西']==1:
            df['setsudo_hi_南東']=1
            df['setsudo_hi_北西']=1
        elif df['setsudo_hi_南東＋南西']==1:
            df['setsudo_hi_南東']=1
            df['setsudo_hi_南西']=1
        elif df['setsudo_hi_南東＋東']==1:
            df['setsudo_hi_南東']=1
            df['setsudo_hi_東']=1
        elif df['setsudo_hi_南東＋西']==1:
            df['setsudo_hi_南東']=1
            df['setsudo_hi_西']=1
        elif df['setsudo_hi_南西＋北']==1:
            df['setsudo_hi_南西']=1
            df['setsudo_hi_北']=1
        elif df['setsudo_hi_南西＋北']==1:
            df['setsudo_hi_南西']=1
            df['setsudo_hi_北']=1
        elif df['setsudo_hi_南西＋北東']==1:
            df['setsudo_hi_南西']=1
            df['setsudo_hi_北東']=1
        elif df['setsudo_hi_南西＋北西']==1:
            df['setsudo_hi_南西']=1
            df['setsudo_hi_北西']=1
        elif df['setsudo_hi_南西＋南東']==1:
            df['setsudo_hi_南西']=1
            df['setsudo_hi_南東']=1
        elif df['setsudo_hi_南西＋北']==1:
            df['setsudo_hi_南西']=1
            df['setsudo_hi_北']=1
        elif df['setsudo_hi_南西＋東']==1:
            df['setsudo_hi_南西']=1
            df['setsudo_hi_東']=1
        elif df['setsudo_hi_南西＋西']==1:
            df['setsudo_hi_南西']=1
            df['setsudo_hi_西']=1
        elif df['setsudo_hi_南＋北']==1:
            df['setsudo_hi_南']=1
            df['setsudo_hi_北']=1
        elif df['setsudo_hi_南＋北東']==1:
            df['setsudo_hi_南']=1
            df['setsudo_hi_北東']=1
        elif df['setsudo_hi_南＋北西']==1:
            df['setsudo_hi_南']=1
            df['setsudo_hi_北西']=1
        elif df['setsudo_hi_南＋南東']==1:
            df['setsudo_hi_南']=1
            df['setsudo_hi_南東']=1
        elif df['setsudo_hi_南＋南西']==1:
            df['setsudo_hi_南']=1
            df['setsudo_hi_南西']=1
        elif df['setsudo_hi_南＋東']==1:
            df['setsudo_hi_南']=1
            df['setsudo_hi_東']=1
        elif df['setsudo_hi_南＋西']==1:
            df['setsudo_hi_南']=1
            df['setsudo_hi_西']=1
        elif df['setsudo_hi_東＋北']==1:
            df['setsudo_hi_東']=1
            df['setsudo_hi_北']=1
        elif df['setsudo_hi_東＋南']==1:
            df['setsudo_hi_東']=1
            df['setsudo_hi_南']=1
        elif df['setsudo_hi_東＋南東']==1:
            df['setsudo_hi_東']=1
            df['setsudo_hi_南東']=1
        elif df['setsudo_hi_東＋南西']==1:
            df['setsudo_hi_東']=1
            df['setsudo_hi_南西']=1
        elif df['setsudo_hi_東＋西']==1:
            df['setsudo_hi_東']=1
            df['setsudo_hi_西']=1
        elif df['setsudo_hi_西＋北']==1:
            df['setsudo_hi_西']=1
            df['setsudo_hi_北']=1
        elif df['setsudo_hi_西＋北東']==1:
            df['setsudo_hi_西']=1
            df['setsudo_hi_北東']=1
        elif df['setsudo_hi_西＋北西']==1:
            df['setsudo_hi_西']=1
            df['setsudo_hi_北西']=1
        elif df['setsudo_hi_西＋東']==1:
            df['setsudo_hi_西']=1
            df['setsudo_hi_東']=1
        elif df['setsudo_hi_西＋南東']==1:
            df['setsudo_hi_西']=1
            df['setsudo_hi_南東']=1
        elif df['setsudo_hi_西＋南西']==1:
            df['setsudo_hi_西']=1
            df['setsudo_hi_南西']=1
        elif df['setsudo_hi_西＋東']==1:
            df['setsudo_hi_西']=1
            df['setsudo_hi_東']=1
        return



In [30]:
df.at[10,'setsudo_hi_北東＋北西']

0

In [29]:
df.apply(process_setsudo_hi)

KeyError: ('setsudo_hi_北東＋北西', 'occurred at index id')

### 検証用のコードを書く

In [2]:
import numpy as np
def mean_absolute_percentage_error( y_train: np.array, y_pred: np.array):
    diff = 0
    n = len(y_train)
    for i in range(n):
        diff += abs(y_train[i]-y_pred[i])/y_train[i]
    score = 100*diff / n
    
    return score

### 6/30 V5向けのトライアル

In [1]:
# tt_msekiが第３位だが、欠損がtrain/test双方とも0.5%程度ある。⇒容積率ごとの平均のtc_msekiとtt_msekiの比から欠損を補完したい
import pandas as pd
import numpy as np
genba = pd.read_csv("data/train_genba.tsv", sep='\t')
goto = pd.read_csv("data/train_goto.tsv",sep='\t')
def create_list( df,cols ):
    names = set()
    for col in cols:
        for i in range(len(df)):
            name = df.at[i,col]
            if ( type(name) != float ):
                names.add(name)
    return sorted(list(names))

In [2]:
jgoto = goto.merge(genba, how='left', on='pj_no' )

In [3]:
jgoto.to_csv("data/joinded_goto.csv",index=False)

In [5]:
df = jgoto.copy()

モデルの`feature_importances_`を確認すると、以下の改善の余地があることがわかった。
- tt_msekiが第３位だが、欠損がtrain/test双方とも0.5%程度ある。
    - genba/gotoを結合後、genbaのtt_mseki_avg_hbでtt_msekiを補完する。
    - genbaのtt_mseki_min_hbとtt_mseki_max_hbの平均値で補完する
    - 容積率80/100/150/200/300/400で欠損が残るので、それぞれの容積率の平均の対建物比率で算出

In [6]:
# 最初の二つの項目は一つのラムダ式
df['new_tt_mseki'] = df.apply(lambda x : x['tt_mseki'] if x['tt_mseki'] != 0 else x['tt_mseki_avg_hb'] if x['tt_mseki_avg_hb']!= 0 else x['tt_mseki_max_hb']  if x['tt_mseki_max_hb'] != 0 else 0, axis=1)

In [7]:
# 1-1
df2 = df[df['new_tt_mseki']!=0]

In [11]:
# 1-2
yoseki_gr = df2.groupby('yoseki1')

In [12]:
ratio_dict = {}
for y in yoseki_gr.groups.keys():
    # 1-3
    d = yoseki_gr.get_group(y)
    #1-4
    total_tc = d['tc_mseki'].sum()
    total_tt = d['tt_mseki'].sum()
    ratio = total_tt/total_tc
    #1-5
    ratio_dict[y]= ratio


In [17]:
def calc_tt_mseki( x ):
    if x['tt_mseki'] != 0:
        return x['tt_mseki']
    yoseki = x['yoseki1']
    ratio = ratio_dict[yoseki]
    tt_mseki = x['tc_mseki'] * ratio
    return tt_mseki

In [25]:
df['new_tt_mseki'] = df.apply(calc_tt_mseki, axis=1)

In [27]:
(df['new_tt_mseki']==0).sum()

0

In [28]:
df.to_csv("data/tmp.csv")

In [ ]:
g = tmp.groupby('yoseki1')

In [ ]:
df = jgoto.copy()

In [ ]:
df['tt_mseki_pro'] = df.apply(lambda x : x['tt_mseki'] if x['tt_mseki'] != 0 else x['tt_mseki_avg_hb'] if x['tt_mseki_avg_hb']!= 0 else x['tt_mseki_max_hb']  if x['tt_mseki_max_hb'] != 0 else 0, axis=1)

In [ ]:
(df['tt_mseki_pro']==0).sum()

In [ ]:
y80_avg = g.get_group(80)

In [ ]:
type(y80_avg)

In [ ]:
g

## 6/29 submit1.tsvで使ったRandomForestでは、変数の重要度はどうだったのだろうか
今保留になっている号棟データの接道方位や現場データの道路の方位が上位なら、やはり実装が必要と言うことになる

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
model = joblib.load('data/randomforest_r1.model')
x_train = pd.read_csv('data/processed_train_goto_x.csv')

In [ ]:
x_train = x_train.drop(['id','pj_no'],axis=1)
importance = pd.DataFrame(model.feature_importances_, index=x_train.columns)
importance.to_csv("data/importance_submit1.csv")

⇒importance_submit1にコメント記入した。

課題で使うことになっている評価関数(Mean Absolute Percentage Error)をGridSearchCVで使うため、評価関数を自作

In [ ]:
import numpy as np
def mean_absolute_percentage_error( y_train: np.array, y_pred: np.array):
    diff = 0
    n = len(y_train)
    for i in range(n):
        diff += abs(y_train[i]-y_pred[i])/y_train[i]
    score = 100*diff / n
    
    return score

In [ ]:
t = np.array([100,100,100,100,100])
p = np.array([90,110,100,100,100])

In [ ]:
print (mean_absolute_percentage_error(t,p))

trainとtestのデータから、同じダミー変数列を生成するためのコード

In [ ]:
def create_list( df1, df2, cols ):
    names = set()
    for col in cols:
        for i in range(len(df1)):
            name = df1.at[i,col]
            if ( type(name) != float ):
                names.add(name)
        for i in range(len(df2)):
            name = df2.at[i,col]
            if ( type(name) != float ):
                names.add(name)
    return sorted(list(names))

In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv("data/train_genba.tsv",sep='\t')
test = pd.read_csv("data/test_genba.tsv",sep='\t')
hokakisei_list = create_list(train, test, ['hokakisei1','hokakisei2','hokakisei3','hokakisei4'])

In [ ]:
print(hokakisei_list)

In [ ]:
train = pd.read_csv("data/train_goto.tsv",sep='\t')
test = pd.read_csv("data/train_goto.tsv",sep='\t')
kobetsu_list = create_list( train, test, ['kobetsu1','kobetsu2','kobetsu3','kobetsu4'])
print(kobetsu_list)

### 現場データのhokakisei1～4から出現するデータをリストにして返す

### 号棟データのkobetsu1～4をダミー変数化する

In [ ]:
def put_name(col, name):
    if ( type(name) != float ):
        col.add(name)

import numpy as np
import pandas as pd
import math
train_goto = pd.read_csv("data/train_goto.tsv",sep='\t')
df = train_goto.copy()

cols = set()
for i in range(len(df)):
    put_name(cols, df.at[i,'kobetsu1'])
    put_name(cols, df.at[i,'kobetsu2'])
    put_name(cols, df.at[i,'kobetsu3'])
    put_name(cols, df.at[i,'kobetsu4'])
    
kobetsu_map = {'信号近い':'信号前'}
df['kobetsu1']=df['kobetsu1'].replace(kobetsu_map)
df['kobetsu2']=df['kobetsu2'].replace(kobetsu_map)
df['kobetsu3']=df['kobetsu3'].replace(kobetsu_map)
df['kobetsu4']=df['kobetsu4'].replace(kobetsu_map)

# ステップ２：
kobetsu_dict = {}
for x in cols:
    kobetsu_dict[x] = np.zeros(len(df), dtype=np.int)

# ステップ３：
for i in range(len(df)):
    name = df.at[i,'kobetsu1']
    if ( type(name) != float):
        kobetsu= kobetsu_dict[name]
        kobetsu[i]=1
    name = df.at[i,'kobetsu2']
    if ( type(name) != float):
        kobetsu= kobetsu_dict[name]
        kobetsu[i]=1
    name = df.at[i,'kobetsu3']
    if ( type(name) != float):
        kobetsu= kobetsu_dict[name]
        kobetsu[i]=1
    name = df.at[i,'kobetsu4']
    if ( type(name) != float):
        kobetsu= kobetsu_dict[name]
        kobetsu[i]=1

# 元のDataFrameとダミー変数化したものを連結
new_genba = pd.concat([df, pd.DataFrame(kobetsu_dict)], axis=1)
new_genba.to_csv("data/temp.csv")

# 不要になったkobetsu1～kobestu4を削除


In [ ]:
cols

### RandomForestを使って特徴量の重要度を求める

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
x_train = pd.read_csv('data/processed_train_goto.csv')
Y_train = pd.DataFrame(x_train['tc_tanka'])
X_train = x_train.drop(['id','pj_no','tc_tanka'],axis=1)

In [ ]:
model = RandomForestRegressor()
model.fit(X_train.values, Y_train.values.ravel())

In [ ]:
print(X_train.columns)

In [ ]:
importance = pd.DataFrame(model.feature_importances_.reshape(1,len(model.feature_importances_)), columns=X_train.columns)
importance.to_csv("data/importance1.csv")
# ⇒　横に長い一覧表になる

In [ ]:
importance = pd.DataFrame(model.feature_importances_, index=X_train.columns)
importance.to_csv("data/importance2.csv")
# ⇒縦に長い一覧表になる

### 四つの列からなる法規制列のダミー変数化を行う
ダミー変数化の前には、同一だが違う名前のついているデータを一意にする


In [ ]:
import numpy as np
import pandas as pd
import math
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')

#### 統一が必要なカラム
- 公拡法　と　公有地拡大推進法　⇒　公拡法へ
- 埋蔵文化財　と　文化財保護法　と　文化財保護法（埋蔵文化財）　⇒　埋蔵文化財へ
- 景観法　と　景観地区　⇒　景観法へ
- 東日本震災復興特　と　東日本大震災復興特別区域法　⇒　東日本震災復興特　へ
- 農地法　と　農地法届出要　⇒　農地法　へ

#### 法規制のうち、出現回数が1,2回のものがある。⇒　出現階数が４以上のものに絞ると次のように。本当に絞るかはRandomForestの重要度を見てから決めることにする。
- 公拡法 あわせて１９回
- 区画整理法 4回
- 国土法　11回
- 埋蔵文化財 75+2+39回
- 景観法 549+23回
- 東日本震災復興特 7+1回
- 河川法 10回
- 自然公園法 9回
- 航空法 19回
- 農地法 270+15回
- 風致地区 １９回

In [ ]:
# ステップ１：　hokaisei1〜4に対して法規制の名称の統一をする 
# ステップ２：対象とする法規制のダミー変数列を作る
# ステップ3：ダミー変数化する

def put_name(col, name):
    if ( type(name) != float ):
        col.add(name)

# ステップ１：
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')
df = train_genba.copy()
kisei_map = {'公有地拡大推進法':'公拡法', '文化財保護法':'埋蔵文化財', '文化財保護法（埋蔵文化財）':'埋蔵文化財', '景観地区':'景観法','東日本大震災復興特別区域法':'東日本震災復興特', '農地法届出要':'農地法　'}
#kisei_map = {'公有地拡大推進法':'公拡法', '文化財保護法':'埋蔵文化財', '文化財保護法（埋蔵文化財）　':'埋蔵文化財','景観地区':'景観法','東日本大震災復興特別区域法':'東日本震災復興特'}

df['hokakisei1']=df['hokakisei1'].replace(kisei_map)
df['hokakisei2']=df['hokakisei2'].replace(kisei_map)
df['hokakisei3']=df['hokakisei3'].replace(kisei_map)
df['hokakisei4']=df['hokakisei4'].replace(kisei_map)

cols = set()
for i in range(len(train_genba)):
    put_name(cols, df.at[i,'hokakisei1'])
    put_name(cols, df.at[i,'hokakisei2'])
    put_name(cols, df.at[i,'hokakisei3'])
    put_name(cols, df.at[i,'hokakisei4'])

# ステップ２：
kisei_dict = {}
for x in cols:
    kisei_dict[x] = np.zeros(len(df), dtype=np.int)

# ステップ３：
for i in range(len(df)):
    name = df.at[i,'hokakisei1']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = df.at[i,'hokakisei2']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = df.at[i,'hokakisei3']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = df.at[i,'hokakisei4']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1

# 元のDataFrameとダミー変数化したものを連結
new_genba = pd.concat([df, pd.DataFrame(kisei_dict)], axis=1)
new_genba.to_csv("data/temp.csv")





In [ ]:
import math

In [ ]:
cols = set()
for i in range(len(train_genba)):
    name = train_genba.at[i,'hokakisei1']
    if ( type(name) != float):
        cols.add(name)
    name = train_genba.at[i,'hokakisei2']
    if ( type(name) != float):
        cols.add(name)
    name = train_genba.at[i,'hokakisei3']
    if ( type(name) != float):
        cols.add(name)
    name = train_genba.at[i,'hokakisei4']
    if ( type(name) != float):
        cols.add(name)

kisei_dict = {}
for x in cols:
    kisei_dict[x] = np.zeros(len(train_genba),dtype=np.int)
    
for i in range(len(train_genba)):
    name = train_genba.at[i,'hokakisei1']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = train_genba.at[i,'hokakisei2']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = train_genba.at[i,'hokakisei3']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1
    name = train_genba.at[i,'hokakisei4']
    if ( type(name) != float):
        kisei = kisei_dict[name]
        kisei[i]=1

kisei_df = pd.DataFrame(kisei_dict)
kisei_df.head()

In [ ]:
 train_genba.tail()

In [ ]:
new_tgenba = pd.concat([train_genba, kisei_df], axis=1)

In [ ]:
new_tgenba.to_csv("data/temp.csv")

In [ ]:
pd.DataFrame(kisei_dict).to_csv("data/temp.csv")

##### 6/22の作業

In [ ]:
df = pd.DataFrame()
df['pj']=prod_goto['pj_no']

In [ ]:
df['tc_mseki'] = prod_goto['tc_mseki']

In [ ]:
df[['tt_mseki', 'fukuin']]=prod_goto[['tt_mseki','fukuin']]

ここからモデル作成と予測をするコードを書く

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb 
from xgboost import XGBClassifier 

In [ ]:
x_train = pd.read_csv('data/processed_train_goto.csv')
Y_train = pd.DataFrame(x_train['tc_tanka'])

In [ ]:
X_train = x_train.drop(['id','pj_no','tc_tanka'],axis=1)
X_train = X_train.values
Y_train = Y_train.values.ravel()

In [ ]:
# XGBoostは時間かかりすぎ
#  XGB = XGBClassifier(n_estimater=500,n_jobs=-1)
# XGB.fit(X_train, Y_train)
#
# ⇒　RandomForestで重要度を見てから、不要な特徴量を落とすことにする
#

In [ ]:
df.head()

#### 6/20の作業

In [ ]:
df = train_genba.copy()
df['gk_yoc_tm'] = df['gk_yoc_tm'].fillna(df['gk_yoc_tm'].mean())

In [ ]:
pd.DataFrame(prod_goto.isnull().sum()).to_csv("data/null_check.csv")

In [ ]:
import numpy as np
import pandas as pd
test_goto = pd.read_csv("data/processed_test_goto.csv")
pd.DataFrame(test_goto.isnull().sum()).to_csv("data/null_check_test.csv")

#### 確認の結果、次のカラムにNULLを含む
#### chiseki_kb_hb,kaoku_hb, mseki_rd_hb, mseki_dp_hb, chiseki_diff

In [ ]:
    # 間口(magutchi)
    # 幅員と同じように補完する。
    df['magutchi'] = df['magutchi'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'magutchi'] ==0 ) :
            df.at[i,'magutchi'] = df.at[i,'road1_mg']    

#### 6/18,19の作業

In [ ]:
df = train_goto.copy()

In [ ]:
genba = pd.read_csv("data/processed_train_genba.csv")

In [ ]:
newdf=df.merge(genba, how="left", on='pj_no')

In [ ]:
newdf.to_csv("data/joined_data.csv")
df = newdf

In [ ]:
df['fukuin'] = df['fukuin'].fillna(0)
for i in range(len(df)):
    if ( df.at[i,'fukuin'] ==0 ) :
        df.at[i,'fukuin'] = df.at[i,'road1_fi']

In [ ]:
    df['magutchi'] = df['magutchi'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'magutchi'] ==0 ) :
            df.at[i,'magutchi'] = df.at[i,'road1_mg']    

In [ ]:
df['setsudo_hi' ] = df['setsudo_hi'].fillna('unknown')
for i in range(len(df)):
    if ( df.at[i,'setsudo_hi']=='unknown'):
        df.at[i.'setsudo_hi']=df.at[i,'road1_hk']

In [12]:
## 元データでjoinしておく
import pandas as pd
genba = pd.read_csv("data/train_genba.tsv", sep='\t')
goto = pd.read_csv("data/train_goto.tsv", sep='\t')
df = goto.merge(genba, how="left", on='pj_no')
df.to_csv("data/org_joined_data.csv")

In [11]:
df.head()

,id,pj_no,keiyaku_pr,tc_mseki,tt_mseki,levelplan,fukuin,road_st,magutchi,setsudo_hi,...,bas_toho1,eki_kyori1,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2
0,train_0000,0,39800000,109.26,104.43,2F/4LDK,4.0,問題なし,9.9,東,...,徒歩,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,train_0001,1,22300000,136.11,105.16,2F/4LDK,4.2,問題なし,9.3,北西,...,徒歩,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,train_0002,2,19800000,87.07,85.94,2F/4LDK,4.0,問題なし,11.1,東,...,徒歩,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,train_0003,3,33990000,163.75,105.98,2F/4LDK,4.0,問題なし,10.5,南,...,徒歩,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,train_0004,4,30800000,111.51,89.01,2F/4LDK,4.0,問題なし,13.5,北,...,徒歩,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.loc[146,'fukuin']

In [ ]:
df['fukuin'] = df['fukuin'].fillna(0)

In [ ]:
df['fukuin'].isnull().sum()

#### いつのだかわからないコード

In [ ]:
df = train_genba.copy()

In [ ]:
df[['bus', 'walk']] = df.apply(calculateLogistics, axis=1)

In [ ]:
df.loc[0:20,['bas_toho1','eki_kyori1','teiho1','bus','walk']]

#### 下記は6/16夜の作業

In [ ]:
def calculateLogistics(x):
    if ( x['bas_toho1'] == 'バス' ):
        bus = x['eki_kyori1']
        walk = x['teiho1']
    else:
        bus = 0
        walk = x['eki_kyori1']
    
    return pd.Series([bus, walk])
    

#### 下記は6/16の作業

In [ ]:
df[['yoto','kempei','yoseki']] = df.apply(select_yoto, axis=1)

In [ ]:
np.isnan(df.loc[0,'yoto2'])

In [ ]:
type(df.loc[26,'yoto2'])==str

In [ ]:
df.loc[20,'shu_soon']

In [ ]:
df['shu_soon'].map({'○':1, np.NaN:0})

In [ ]:
def select_yoto(x):
    if ( type(x['yoto2']) == str ):
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    else:
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1']
    """
    if ( np.isnan(x['yoto2'])):
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    """
    """"
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    """
    return pd.Series([yoto, kempei, yoseki])


#### 下記は6/15の作業

In [ ]:
df = df.drop('jukyo',axis=1)

In [ ]:
# df['yoto'], df['kempei'],df['yoseki']= df.apply(select_yoto,axis=1)
df[['yoto','kempei','yoseki']] = df.apply(select_yoto,axis=1)

In [ ]:
df.head()

In [ ]:
org_df = df.copy()

In [ ]:
df['josui'] = df.apply(lambda x: 1 if x['josui']=='公営' else 0, axis=1)

In [ ]:
df['gesui'] = df.apply(lambda x: 1 if x['gesui']=='公共下水' else 0, axis=1)

In [ ]:
df['gas'] = df.apply(lambda x: 2 if x['gas'] =='都市ガス' else 1 if x['gas'] == '集中プロパン' else 0, axis=1)

In [ ]:
df['chiseki_diff'] = df['chiseki_kb_hb']-df['chiseki_js_hb']

In [ ]:
a = pd.get_dummies(df[['usui']],drop_first=False)

In [ ]:
df2 = pd.merge(df, a, left_index=True, right_index=True)

In [ ]:
df = pd.merge(df, pd.get_dummies(df[['usui']],drop_first = False), left_index = True, right_index = True)

In [ ]:
df = df.fillna({'tateuri_su': 0})

In [ ]:
df = pd.merge(df, pd.get_dummies(df[['road1_hk','road1_sb']],drop_first = False), left_index = True, right_index = True)

In [ ]:
df['road_num']=df.apply(lambda x: 4 if x['road4_hk']!=np.nan else 3)

In [ ]:
df['road_num']=df.apply(lambda x: 4 if x['road4_fi']!=0 else 3 if x['road3_fi']!=0 else 2 if x['road2_fi']!=0 else 1, axis=1)

In [ ]:
import math

In [ ]:
df['road_num']=df.apply(lambda x: 4 if not math.isnan(x['road4_fi']) else 3 if not math.isnan(x['road3_fi']) else 2 if not math.isnan(x['road2_fi']) else 1, axis=1)

In [ ]:
df['toshikuiki']=df['toshikuiki2'].fillna('empty')

In [ ]:
df['toshikuiki'] = df.apply(lambda x: x['toshikuiki1'] if x['toshikuiki']=="empty" else x['toshikuiki2'], axis=1)

In [ ]:
df['toshikuiki']

In [ ]:
df.loc[1549,'toshikuiki2']

In [ ]:
def select_yoto(x):
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    return pd.Series([yoto, kempei, yoseki])
